In [64]:
import datetime
import requests
import pytz

In [65]:
# Tools
def get_system_time(format: str = "%y-%m-%d %H:%M:%S"):
    """Returns the current date and time in the specified format."""
    current_time = datetime.datetime.now()
    return current_time.strftime(format)

In [66]:
def get_current_location():
    """Returns the current location (city and country) based on the user's IP address."""
    try:
        response = requests.get("https://ipinfo.io")
        data = response.json()
        city = data.get("city", "Unknown City")
        country = data.get("country", "Unknown Country")
        return f"Current location: {city}, {country}"
    except Exception as e:
        return f"Error fetching location: {e}"

In [67]:
def get_current_weather(location: str):
    """Returns the current weather for the specified location."""
    try:
        api_key = "b00cec05979ffc064beabca84270ef62"
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()

        if data.get("cod") != 200:
            return f"Error: {data.get('message', 'Unknown error')}"

        weather = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        return f"Weather in {location}: {weather}, Temperature: {temperature}°C, Humidity: {humidity}%"
    except Exception as e:
        return f"Error fetching weather: {e}"

In [68]:

def get_time_in_timezone(timezone: str, format: str = "%y-%m-%d %H:%M:%S"):
    """Returns the current time in the specified timezone."""
    city_timezone_mapping = {
        "london": "Europe/London",
        "new york": "America/New_York",
        "tokyo": "Asia/Tokyo",
        "sydney": "Australia/Sydney",
        "paris": "Europe/Paris"
    }

    try:
        tz_name = city_timezone_mapping.get(timezone.lower().strip(), timezone)
        tz = pytz.timezone(tz_name)
        current_time = datetime.datetime.now(tz)
        return current_time.strftime(format)
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"

In [69]:

# Message History
class MessageHistory:
    def __init__(self):
        self.messages = []

    def add_message(self, sender, content):
        self.messages.append({"sender": sender, "content": content})

    def display(self):
        for message in self.messages:
            print(f"{message['sender'].upper()}: {message['content']}")

In [70]:

# Extract location from query
def extract_location(query, keyword):
    match = re.search(rf"{keyword} in ([a-zA-Z\s]+)[?.!]*", query, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return None

In [71]:

# Agent Logic
def agent(query):
    tools = {
        "current time": get_system_time,
        "current location": get_current_location,
        "weather": get_current_weather,
        "timezone": get_time_in_timezone
    }

    if "current time in" in query.lower():
        location = extract_location(query, "current time")
        if location:
            return tools["timezone"](timezone=location)
    elif "current time" in query.lower():
        return tools["current time"]()
    elif "current location" in query.lower():
        return tools["current location"]()
    elif "weather in" in query.lower():
        location = extract_location(query, "weather")
        if location:
            return tools["weather"](location)

    return "I'm not sure how to handle that request."

In [72]:

# Query Execution
message_history = MessageHistory()
query = "What is the current time in London?"
message_history.add_message("user", query)

response = agent(query)
message_history.add_message("agent", response)

# Displaying conversation history
message_history.display()


USER: What is the current time in London?
AGENT: 25-02-04 02:40:04
